# Aggregating counts and rates of hospital admissions and make a table

In [1]:
import pandas as pd
import numpy as np
import functools as ft

## 0- functions and read data

In [2]:
pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_deciles_lrti_incdt.csv')

,variable,count,mean,std
0,"bins_(0.0012499999999999998, 0.00463]",16025.0,0.100004,0.300007
1,"bins_(0.00463, 0.00514]",16025.0,0.099998,0.299998
2,"bins_(0.00514, 0.00555]",16025.0,0.099998,0.299998
3,"bins_(0.00555, 0.00592]",16025.0,0.099998,0.299998
4,"bins_(0.00592, 0.00629]",16025.0,0.100004,0.300007
5,"bins_(0.00629, 0.00668]",16025.0,0.099998,0.299998
6,"bins_(0.00668, 0.00712]",16025.0,0.099998,0.299998
7,"bins_(0.00712, 0.00767]",16025.0,0.100004,0.300007
8,"bins_(0.00767, 0.00852]",16025.0,0.099992,0.299990
9,"bins_(0.00852, 0.0191]",16025.0,0.100004,0.300007


In [ ]:
#read summary table of deciles
summary_table_deciles_lrti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_deciles_lrti_incdt.csv')
summary_table_deciles_lrti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_deciles_lrti_prevl.csv')
summary_table_deciles_urti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_deciles_urti_incdt.csv')
summary_table_deciles_urti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_deciles_urti_prevl.csv')
summary_table_deciles_uti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_deciles_uti_incdt.csv')
summary_table_deciles_uti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_deciles_uti_prevl.csv')
summary_table_deciles_sinusitis_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_deciles_sinusitis_incdt.csv')
summary_table_deciles_sinusitis_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_deciles_sinusitis_prevl.csv')
summary_table_deciles_otmedia_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_deciles_otmedia_incdt.csv')
summary_table_deciles_otmedia_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_deciles_otmedia_prevl.csv')
summary_table_deciles_ot_externa_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_deciles_ot_externa_incdt.csv')
summary_table_deciles_ot_externa_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_deciles_ot_externa_prevl.csv')
summary_table_deciles_urti_combined_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_deciles_urti_combined_incdt.csv')
summary_table_deciles_urti_combined_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_deciles_urti_combined_prevl.csv')
summary_table_deciles_cough_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_deciles_cough_incdt.csv')
summary_table_deciles_cough_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_deciles_cough_prevl.csv')
summary_table_deciles_cough_cold_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_deciles_cough_cold_incdt.csv')
summary_table_deciles_cough_cold_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_deciles_cough_cold_prevl.csv')
summary_table_deciles_throat_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_deciles_throat_incdt.csv')
summary_table_deciles_throat_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_deciles_throat_prevl.csv')

In [2]:
def data_process(data, data_hosped, infection):
    #replace SMs with 5 in data and data_hosped and NaN with 0 in data_hosped 
    data = data.replace({'SM': 5})
    data_hosped = data_hosped.replace({'SM': 5, np.nan:0})
    #conver counts into int
    data[infection] = data[infection].astype(str).replace('\.0', '', regex=True).astype(int)
    data_hosped[infection] = data_hosped[infection].astype(str).replace('\.0', '', regex=True).astype(int)

    #add hosp rate
    data_hosped[infection+'_rate'] = (data_hosped[infection]/data[infection])*1000
    data_hosped[infection+'_rate'] = np.round(data_hosped[infection+'_rate'], decimals=1)

    #replace rate with SN if infection count equals 5
    data_hosped.loc[data_hosped[infection] == 5, infection+'_rate'] = 'SN'

    #calculate total using sex_female and sex_male
    total = data_hosped.iloc[int(data_hosped[data_hosped['variable']=='sex_male'].index[0])][infection] + data_hosped.iloc[int(data_hosped[data_hosped['variable']=='sex_female'].index[0])][infection]
    total = "{:,}".format(total)
    data_hosped['variable'] = data_hosped['variable'].str.replace('antibacterial_brit', 'total')
    
    #add comma to thousands of column count
    data_hosped[infection] = data_hosped[infection].apply(lambda x : "{:,}".format(x))
    #make new column with counts and rates
    data_hosped[infection+'_rate_count'] = data_hosped[infection].astype(str) + ' (' + data_hosped[infection+'_rate'].astype(str) + ')'
    
    #replace total (after making new column of counts and rates)
    data_hosped.at[int(data_hosped[data_hosped['variable']=='total'].index[0]),infection+'_rate_count'] = total

    #drop redundant columns and rename the needed column
    data_hosped = data_hosped.drop([infection, infection+'_rate'], axis=1)
    data_hosped = data_hosped.rename(columns={infection+'_rate_count': infection})
    #replace any row with no value (=0) with a dash 
    data_hosped[infection] = data_hosped[infection].replace({'0 (0.0)': '-'})
    #replace count and rate if rate contains SN
    data_hosped.loc[data_hosped[infection].str.contains('SN', regex=False), infection] = 'SN' 

    return data_hosped[[infection, 'variable']]

In [3]:
summary_table_cases_lrti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_incdt_cases.csv')
summary_table_cases_lrti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_prevl_cases.csv')
summary_table_cases_urti_combined_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_incdt_cases.csv')
summary_table_cases_urti_combined_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_prevl_cases.csv')
summary_table_cases_uti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_incdt_cases.csv')
summary_table_cases_uti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_prevl_cases.csv')
summary_table_cases_sinusitis_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_incdt_cases.csv')
summary_table_cases_sinusitis_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_prevl_cases.csv')
summary_table_cases_otmedia_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_incdt_cases.csv')
summary_table_cases_otmedia_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_prevl_cases.csv')
summary_table_cases_ot_externa_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_incdt_cases.csv')
summary_table_cases_ot_externa_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_prevl_cases.csv')
summary_table_cases_urti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_incdt_cases.csv')
summary_table_cases_urti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_prevl_cases.csv')
summary_table_cases_cough_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_incdt_cases.csv')
summary_table_cases_cough_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_prevl_cases.csv')
summary_table_cases_cough_cold_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_incdt_cases.csv')
summary_table_cases_cough_cold_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_prevl_cases.csv')
summary_table_cases_throat_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_incdt_cases.csv')
summary_table_cases_throat_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_prevl_cases.csv')

In [4]:
summary_table_lrti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_incdt.csv')
summary_table_lrti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_prevl.csv')
summary_table_urti_combined_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_incdt.csv')
summary_table_urti_combined_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_prevl.csv')
summary_table_uti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_incdt.csv')
summary_table_uti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_prevl.csv')
summary_table_sinusitis_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_incdt.csv')
summary_table_sinusitis_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_prevl.csv')
summary_table_otmedia_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_incdt.csv')
summary_table_otmedia_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_prevl.csv')
summary_table_ot_externa_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_incdt.csv')
summary_table_ot_externa_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_prevl.csv')
summary_table_urti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_incdt.csv')
summary_table_urti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_prevl.csv')
summary_table_cough_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_incdt.csv')
summary_table_cough_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_prevl.csv')
summary_table_cough_cold_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_incdt.csv')
summary_table_cough_cold_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_prevl.csv')
summary_table_throat_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_incdt.csv')
summary_table_throat_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_prevl.csv')

In [5]:
#select relevant columns of each df
summary_table_lrti_incdt = summary_table_lrti_incdt[['variable', 'count']]
summary_table_lrti_prevl = summary_table_lrti_prevl[['variable', 'count']]
summary_table_urti_combined_incdt = summary_table_urti_combined_incdt[['variable', 'count']]
summary_table_urti_combined_prevl = summary_table_urti_combined_prevl[['variable', 'count']]
summary_table_uti_incdt = summary_table_uti_incdt[['variable', 'count']]
summary_table_uti_prevl = summary_table_uti_prevl[['variable', 'count']]
summary_table_sinusitis_incdt = summary_table_sinusitis_incdt[['variable', 'count']]
summary_table_sinusitis_prevl = summary_table_sinusitis_prevl[['variable', 'count']]
summary_table_otmedia_incdt = summary_table_otmedia_incdt[['variable', 'count']]
summary_table_otmedia_prevl = summary_table_otmedia_prevl[['variable', 'count']]
summary_table_ot_externa_incdt = summary_table_ot_externa_incdt[['variable', 'count']]
summary_table_ot_externa_prevl = summary_table_ot_externa_prevl[['variable', 'count']]
summary_table_urti_incdt = summary_table_urti_incdt[['variable', 'count']]
summary_table_urti_prevl = summary_table_urti_prevl[['variable', 'count']]
summary_table_cough_incdt = summary_table_cough_incdt[['variable', 'count']]
summary_table_cough_prevl = summary_table_cough_prevl[['variable', 'count']]
summary_table_cough_cold_incdt = summary_table_cough_cold_incdt[['variable', 'count']]
summary_table_cough_cold_prevl = summary_table_cough_cold_prevl[['variable', 'count']]
summary_table_throat_incdt = summary_table_throat_incdt[['variable', 'count']]
summary_table_throat_prevl = summary_table_throat_prevl[['variable', 'count']]

In [6]:
#merge baseline dfs
dfs_tab = [summary_table_lrti_incdt, summary_table_lrti_prevl, 
        summary_table_urti_combined_incdt, summary_table_urti_combined_prevl, 
        summary_table_uti_incdt, summary_table_uti_prevl, 
        summary_table_sinusitis_incdt, summary_table_sinusitis_prevl, 
        summary_table_otmedia_incdt, summary_table_otmedia_prevl, 
        summary_table_ot_externa_incdt, summary_table_ot_externa_prevl, 
        summary_table_urti_incdt, summary_table_urti_prevl, 
        summary_table_cough_incdt, summary_table_cough_prevl, 
        summary_table_cough_cold_incdt, summary_table_cough_cold_prevl, 
        summary_table_throat_incdt, summary_table_throat_prevl, 
        ]

#replace any column that contains ab_date with ab_binray
for i in dfs_tab:
    i.loc[i['variable'].str.contains('_ab_date'), 'variable'] = 'ab_binary'

#concat dfs and assign column names
df_concated = pd.concat([i.set_index('variable') for i in dfs_tab],axis=1, join='outer').reset_index()
df_concated.columns = ['variable', 
                        'lrti_incdt', 'lrti_prevl', 
                        'urti_combined_incdt', 'urti_combined_prevl',
                        'uti_incdt', 'uti_prevl',
                        'sinusitis_incdt', 'sinusitis_prevl',
                        'otmedia_incdt', 'otmedia_prevl',
                        'ot_externa_incdt', 'ot_externa_prevl',
                        'urti_incdt', 'urti_prevl',
                        'cough_incdt', 'cough_prevl',
                        'cough_cold_incdt', 'cough_cold_prevl',
                        'throat_incdt', 'throat_prevl'
                        ]

#custom sort baseline dfs
df_concated['variable'] = pd.Categorical(df_concated['variable'], ['age_cat_15_24', 'age_cat_25_34', 'age_cat_35_44', 'age_cat_45_54', 'age_cat_55_64', 'age_cat_65_74', 'age_cat_75_more',
                                                                   'sex_female', 'sex_male',
                                                                   'CCI_cat_high', 'CCI_cat_low', 'CCI_cat_medium', 'CCI_cat_very_high', 'CCI_cat_very_low',
                                                                   'flu_vaccine_no', 'flu_vaccine_yes',
                                                                   'bmi_cat_healthy_weight', 'bmi_cat_obese', 'bmi_cat_overweight', 'bmi_cat_underweight', 'bmi_cat_unknown',
                                                                   'region_east', 'region_east_midlands', 'region_london', 'region_north_east', 'region_north_west', 'region_south_east', 'region_south_west', 'region_west_midlands', 'region_yorkshire',
                                                                   'imd_affluent', 'imd_medium', 'imd_unaffluent', 'imd_unknown', 'imd_very_affluent', 'imd_very_unaffluent',
                                                                   'ethnicity_asian', 'ethnicity_other', 'ethnicity_unknown', 'ethnicity_white',
                                                                   'smoking_ex_smoker', 'smoking_never_smoked', 'smoking_smoker', 'smoking_unknown',
                                                                   'season_autumn', 'season_spring', 'season_summer', 'season_winter',
                                                                   'period_during_pandemic', 'period_post_2nd_lockdown', 'period_prepandemic',
                                                                   'ab_binary', 'antibacterial_brit'])

df_final_tab = df_concated.sort_values("variable").reset_index(drop=True)
# df_final_tab

In [7]:
#select relevant columns of each df
summary_table_cases_lrti_incdt = summary_table_cases_lrti_incdt[['variable', 'count']]
summary_table_cases_lrti_prevl = summary_table_cases_lrti_prevl[['variable', 'count']]
summary_table_cases_urti_combined_incdt = summary_table_cases_urti_combined_incdt[['variable', 'count']]
summary_table_cases_urti_combined_prevl = summary_table_cases_urti_combined_prevl[['variable', 'count']]
summary_table_cases_uti_incdt = summary_table_cases_uti_incdt[['variable', 'count']]
summary_table_cases_uti_prevl = summary_table_cases_uti_prevl[['variable', 'count']]
summary_table_cases_sinusitis_incdt = summary_table_cases_sinusitis_incdt[['variable', 'count']]
summary_table_cases_sinusitis_prevl = summary_table_cases_sinusitis_prevl[['variable', 'count']]
summary_table_cases_otmedia_incdt = summary_table_cases_otmedia_incdt[['variable', 'count']]
summary_table_cases_otmedia_prevl = summary_table_cases_otmedia_prevl[['variable', 'count']]
summary_table_cases_ot_externa_incdt = summary_table_cases_ot_externa_incdt[['variable', 'count']]
summary_table_cases_ot_externa_prevl = summary_table_cases_ot_externa_prevl[['variable', 'count']]
summary_table_cases_urti_incdt = summary_table_cases_urti_incdt[['variable', 'count']]
summary_table_cases_urti_prevl = summary_table_cases_urti_prevl[['variable', 'count']]
summary_table_cases_cough_incdt = summary_table_cases_cough_incdt[['variable', 'count']]
summary_table_cases_cough_prevl = summary_table_cases_cough_prevl[['variable', 'count']]
summary_table_cases_cough_cold_incdt = summary_table_cases_cough_cold_incdt[['variable', 'count']]
summary_table_cases_cough_cold_prevl = summary_table_cases_cough_cold_prevl[['variable', 'count']]
summary_table_cases_throat_incdt = summary_table_cases_throat_incdt[['variable', 'count']]
summary_table_cases_throat_prevl = summary_table_cases_throat_prevl[['variable', 'count']]

In [8]:
#merge hosped dfs
dfs_tab_cases = [summary_table_cases_lrti_incdt, summary_table_cases_lrti_prevl,  
                summary_table_cases_urti_combined_incdt, summary_table_cases_urti_combined_prevl, 
                summary_table_cases_uti_incdt, summary_table_cases_uti_prevl, 
                summary_table_cases_sinusitis_incdt, summary_table_cases_sinusitis_prevl, 
                summary_table_cases_otmedia_incdt, summary_table_cases_otmedia_prevl, 
                summary_table_cases_ot_externa_incdt, summary_table_cases_ot_externa_prevl, 
                summary_table_cases_urti_incdt, summary_table_cases_urti_prevl, 
                summary_table_cases_cough_incdt, summary_table_cases_cough_prevl, 
                summary_table_cases_cough_cold_incdt, summary_table_cases_cough_cold_prevl, 
                summary_table_cases_throat_incdt, summary_table_cases_throat_prevl, 
                ]

#replace any column that contains ab_date with ab_binray
for i in dfs_tab_cases:
    i.loc[i['variable'].str.contains('_ab_date'), 'variable'] = 'ab_binary'

#concat dfs and assign column names
df_concated_cases = pd.concat([j.set_index('variable') for j in dfs_tab_cases],axis=1, join='outer').reset_index()
df_concated_cases.columns = ['variable', 
                        'lrti_incdt', 'lrti_prevl', 
                        'urti_combined_incdt', 'urti_combined_prevl',
                        'uti_incdt', 'uti_prevl',
                        'sinusitis_incdt', 'sinusitis_prevl',
                        'otmedia_incdt', 'otmedia_prevl',
                        'ot_externa_incdt', 'ot_externa_prevl',
                        'urti_incdt', 'urti_prevl',
                        'cough_incdt', 'cough_prevl',
                        'cough_cold_incdt', 'cough_cold_prevl',
                        'throat_incdt', 'throat_prevl'
                        ]

#custom sort factors of variables column 
df_concated_cases['variable'] = pd.Categorical(df_concated_cases['variable'], ['age_cat_15_24', 'age_cat_25_34', 'age_cat_35_44', 'age_cat_45_54', 'age_cat_55_64', 'age_cat_65_74', 'age_cat_75_more',
                                                                                'sex_female', 'sex_male',
                                                                                'CCI_cat_high', 'CCI_cat_low', 'CCI_cat_medium', 'CCI_cat_very_high', 'CCI_cat_very_low',
                                                                                'flu_vaccine_no', 'flu_vaccine_yes',
                                                                                'bmi_cat_healthy_weight', 'bmi_cat_obese', 'bmi_cat_overweight', 'bmi_cat_underweight', 'bmi_cat_unknown',
                                                                                'region_east', 'region_east_midlands', 'region_london', 'region_north_east', 'region_north_west', 'region_south_east', 'region_south_west', 'region_west_midlands', 'region_yorkshire',
                                                                                'imd_affluent', 'imd_medium', 'imd_unaffluent', 'imd_unknown', 'imd_very_affluent', 'imd_very_unaffluent',
                                                                                'ethnicity_asian', 'ethnicity_other', 'ethnicity_unknown', 'ethnicity_white',
                                                                                'smoking_ex_smoker', 'smoking_never_smoked', 'smoking_smoker', 'smoking_unknown',
                                                                                'season_autumn', 'season_spring', 'season_summer', 'season_winter',
                                                                                'period_during_pandemic', 'period_post_2nd_lockdown', 'period_prepandemic',
                                                                                'ab_binary', 'antibacterial_brit'])
df_final_tab_cases = df_concated_cases.sort_values("variable").reset_index(drop=True)
df_final_tab_cases

,variable,lrti_incdt,lrti_prevl,urti_combined_incdt,urti_combined_prevl,uti_incdt,uti_prevl,sinusitis_incdt,sinusitis_prevl,otmedia_incdt,...,ot_externa_incdt,ot_externa_prevl,urti_incdt,urti_prevl,cough_incdt,cough_prevl,cough_cold_incdt,cough_cold_prevl,throat_incdt,throat_prevl
0,age_cat_15_24,140.0,15.0,595.0,90.0,170.0,15.0,145.0,15.0,155.0,...,150.0,20.0,2250.0,235.0,145.0,20.0,145.0,20.0,150.0,20.0
1,age_cat_25_34,180.0,10.0,625.0,70.0,175.0,20.0,145.0,20.0,175.0,...,165.0,10.0,2315.0,265.0,155.0,20.0,155.0,15.0,150.0,20.0
2,age_cat_35_44,145.0,15.0,585.0,55.0,170.0,20.0,185.0,10.0,145.0,...,140.0,15.0,2220.0,270.0,150.0,SM,165.0,15.0,120.0,15.0
3,age_cat_45_54,165.0,25.0,595.0,65.0,155.0,20.0,155.0,15.0,145.0,...,145.0,15.0,2330.0,255.0,150.0,20.0,145.0,15.0,130.0,15.0
4,age_cat_55_64,145.0,20.0,580.0,70.0,125.0,20.0,140.0,20.0,145.0,...,160.0,20.0,2320.0,260.0,140.0,15.0,135.0,25.0,155.0,10.0
5,age_cat_65_74,145.0,25.0,625.0,65.0,155.0,20.0,150.0,10.0,145.0,...,165.0,15.0,2305.0,295.0,160.0,15.0,145.0,15.0,160.0,10.0
6,age_cat_75_more,145.0,15.0,580.0,45.0,160.0,20.0,155.0,15.0,170.0,...,145.0,10.0,2305.0,245.0,165.0,10.0,130.0,20.0,155.0,SM
7,sex_female,550.0,65.0,2130.0,225.0,590.0,60.0,545.0,60.0,580.0,...,545.0,50.0,8175.0,930.0,525.0,50.0,540.0,70.0,520.0,40.0
8,sex_male,515.0,60.0,2055.0,235.0,525.0,75.0,525.0,50.0,500.0,...,530.0,50.0,7870.0,900.0,540.0,60.0,485.0,60.0,495.0,55.0
9,CCI_cat_high,105.0,15.0,455.0,45.0,120.0,10.0,110.0,10.0,95.0,...,105.0,15.0,1695.0,185.0,105.0,10.0,115.0,15.0,115.0,10.0


## 1- process data to make a table of counts and rates, and save it

In [9]:
data_hosped_lrti_incdt = data_process(df_final_tab, df_final_tab_cases, 'lrti_incdt')
data_hosped_lrti_prevl = data_process(df_final_tab, df_final_tab_cases, 'lrti_prevl')
data_hosped_urti_combined_incdt = data_process(df_final_tab, df_final_tab_cases, 'urti_combined_incdt')
data_hosped_urti_combined_prevl = data_process(df_final_tab, df_final_tab_cases, 'urti_combined_prevl')
data_hosped_uti_incdt = data_process(df_final_tab, df_final_tab_cases, 'uti_incdt')
data_hosped_uti_prevl = data_process(df_final_tab, df_final_tab_cases, 'uti_prevl')
data_hosped_sinusitis_incdt = data_process(df_final_tab, df_final_tab_cases, 'sinusitis_incdt')
data_hosped_sinusitis_prevl = data_process(df_final_tab, df_final_tab_cases, 'sinusitis_prevl')
data_hosped_otmedia_incdt = data_process(df_final_tab, df_final_tab_cases, 'otmedia_incdt')
data_hosped_otmedia_prevl = data_process(df_final_tab, df_final_tab_cases, 'otmedia_prevl')
data_hosped_ot_externa_incdt = data_process(df_final_tab, df_final_tab_cases, 'ot_externa_incdt')
data_hosped_ot_externa_prevl = data_process(df_final_tab, df_final_tab_cases, 'ot_externa_prevl')
data_hosped_urti_incdt = data_process(df_final_tab, df_final_tab_cases, 'urti_incdt')
data_hosped_urti_prevl = data_process(df_final_tab, df_final_tab_cases, 'urti_prevl')
data_hosped_cough_incdt = data_process(df_final_tab, df_final_tab_cases, 'cough_incdt')
data_hosped_cough_prevl = data_process(df_final_tab, df_final_tab_cases, 'cough_prevl')
data_hosped_cough_cold_incdt = data_process(df_final_tab, df_final_tab_cases, 'cough_cold_incdt')
data_hosped_cough_cold_prevl = data_process(df_final_tab, df_final_tab_cases, 'cough_cold_prevl')
data_hosped_throat_incdt = data_process(df_final_tab, df_final_tab_cases, 'throat_incdt')
data_hosped_throat_prevl = data_process(df_final_tab, df_final_tab_cases, 'throat_prevl')


dfs_hosped_tab =[data_hosped_lrti_incdt, data_hosped_lrti_prevl, 
                 data_hosped_urti_combined_incdt, data_hosped_urti_combined_prevl,
                 data_hosped_uti_incdt, data_hosped_uti_prevl,
                 data_hosped_sinusitis_incdt, data_hosped_sinusitis_prevl,
                 data_hosped_otmedia_incdt, data_hosped_otmedia_prevl,
                 data_hosped_ot_externa_incdt, data_hosped_ot_externa_prevl,
                 data_hosped_urti_incdt, data_hosped_urti_prevl,
                 data_hosped_cough_incdt, data_hosped_cough_prevl,
                 data_hosped_cough_cold_incdt, data_hosped_cough_cold_prevl,
                 data_hosped_throat_incdt, data_hosped_throat_prevl]

#concat dfs and assign column names
data_hosped = pd.concat([i.set_index('variable') for i in dfs_hosped_tab],axis=1, join='outer').reset_index()

#save hosped count and rate table
data_hosped.to_csv(f'../output/aggregate_cases_table/cases_count_rate_table_all_infections.csv', index=False)
data_hosped

,variable,lrti_incdt,lrti_prevl,urti_combined_incdt,urti_combined_prevl,uti_incdt,uti_prevl,sinusitis_incdt,sinusitis_prevl,otmedia_incdt,...,ot_externa_incdt,ot_externa_prevl,urti_incdt,urti_prevl,cough_incdt,cough_prevl,cough_cold_incdt,cough_cold_prevl,throat_incdt,throat_prevl
0,age_cat_15_24,140 (6.1),15 (5.8),595 (6.6),90 (8.9),170 (7.4),15 (5.9),145 (6.4),15 (5.8),155 (6.7),...,150 (6.4),20 (7.6),"2,250 (99.4)",235 (91.8),145 (6.4),20 (7.9),145 (6.3),20 (7.9),150 (6.7),20 (8.0)
1,age_cat_25_34,180 (7.9),10 (3.9),625 (6.8),70 (6.8),175 (7.7),20 (7.8),145 (6.3),20 (7.9),175 (7.6),...,165 (7.3),10 (4.0),"2,315 (100.4)",265 (101.5),155 (6.7),20 (7.6),155 (6.7),15 (5.8),150 (6.5),20 (8.2)
2,age_cat_35_44,145 (6.3),15 (5.9),585 (6.5),55 (5.4),170 (7.3),20 (8.0),185 (8.0),10 (3.7),145 (6.2),...,140 (6.1),15 (5.9),"2,220 (98.4)",270 (105.3),150 (6.6),SN,165 (7.2),15 (5.9),120 (5.3),15 (6.0)
3,age_cat_45_54,165 (7.2),25 (9.6),595 (6.5),65 (6.5),155 (6.7),20 (7.7),155 (6.8),15 (5.9),145 (6.2),...,145 (6.3),15 (5.9),"2,330 (102.2)",255 (100.4),150 (6.6),20 (8.0),145 (6.3),15 (6.0),130 (5.7),15 (6.0)
4,age_cat_55_64,145 (6.2),20 (7.6),580 (6.3),70 (6.9),125 (5.5),20 (7.7),140 (6.0),20 (7.6),145 (6.2),...,160 (7.0),20 (8.0),"2,320 (102.1)",260 (101.0),140 (6.1),15 (6.0),135 (5.9),25 (9.6),155 (6.8),10 (4.0)
5,age_cat_65_74,145 (6.3),25 (9.3),625 (6.8),65 (6.3),155 (6.7),20 (7.8),150 (6.5),10 (4.0),145 (6.4),...,165 (7.3),15 (5.9),"2,305 (98.9)",295 (112.6),160 (7.0),15 (5.9),145 (6.3),15 (5.9),160 (7.0),10 (3.9)
6,age_cat_75_more,145 (6.3),15 (5.7),580 (6.4),45 (4.4),160 (7.0),20 (7.7),155 (6.8),15 (5.9),170 (7.3),...,145 (6.2),10 (3.8),"2,305 (103.5)",245 (96.5),165 (7.1),10 (4.0),130 (5.8),20 (7.7),155 (6.8),SN
7,sex_female,550 (6.7),65 (7.1),"2,130 (6.6)",225 (6.2),590 (7.2),60 (6.5),545 (6.7),60 (6.5),580 (7.0),...,545 (6.6),50 (5.5),"8,175 (101.1)",930 (100.9),525 (6.4),50 (5.5),540 (6.6),70 (7.6),520 (6.4),40 (4.5)
8,sex_male,515 (6.5),60 (6.6),"2,055 (6.5)",235 (6.8),525 (6.6),75 (8.5),525 (6.7),50 (5.7),500 (6.3),...,530 (6.8),50 (5.7),"7,870 (100.3)",900 (102.4),540 (6.9),60 (7.0),485 (6.1),60 (6.8),495 (6.3),55 (6.4)
9,CCI_cat_high,105 (6.2),15 (8.0),455 (6.7),45 (6.1),120 (7.1),10 (5.1),110 (6.6),10 (5.4),95 (5.7),...,105 (6.2),15 (8.0),"1,695 (101.2)",185 (98.1),105 (6.3),10 (5.4),115 (6.7),15 (8.0),115 (6.8),10 (5.5)
